In [ ]:
import cv2
import numpy as np
from IPython.display import display, clear_output
from PIL import Image
import requests
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import time
from bs4 import BeautifulSoup
import re

# Функция отправки электронных писем
def send_email(subject, message, from_addr, to_addr, password):
    print("Preparing the email...")
    msg = MIMEMultipart()
    msg['From'] = from_addr
    msg['To'] = to_addr
    msg['Subject'] = subject

    msg.attach(MIMEText(message, 'plain'))

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(from_addr, password)
    server.send_message(msg)
    server.quit()
    print("Email sent!")

# Функция отправки уведомления в Telegram
def send_telegram_notification(bot_token, chat_id, message, image_path, x0, y0, x1, y1, motion_objects):
    print("Sending Telegram notification...")
    frame = cv2.imread(image_path)

    # Выделение области интереса красным цветом
    cv2.rectangle(frame, (x0, y0), (x1, y1), (0, 0, 255), 2)

    # Выделение движущихся объектов зеленым цветом
    for motion_object in motion_objects:
        x, y, w, h = motion_object
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imwrite(image_path, frame)

    url = f"https://api.telegram.org/bot{bot_token}/sendPhoto"
    files = {'photo': open(image_path, 'rb')}
    data = {
        'chat_id': chat_id,
        'caption': message
    }
    response = requests.post(url, files=files, data=data)
    if response.status_code == 200:
        print("Telegram notification sent successfully!")
    else:
        print("Failed to send Telegram notification.")

# Определение функции поиска URL видеопотока
def find_video_url(url: str) -> str:
    # Отправка GET-запроса к веб-странице
    response = requests.get(url)
    # Создание объекта BeautifulSoup для разбора HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Нахождение всех скриптов на странице
    scripts = soup.find_all("script")
    video_url = None

    # Обход всех скриптов
    for script in scripts:
        # Поиск подстроки с URL видеопотока с помощью регулярного выражения
        match = re.search(r"//stream\.exdesign\.ru/zilart-16/index\.m3u8\?token=[\w-]+", script.text)
        if match:
            # Если подстрока найдена, добавляем префикс "https:" и сохраняем в переменной video_url
            video_url = "https:" + match.group(0)
            break

    # Возвращаем найденный URL видеопотока или None, если URL не был найден
    return video_url

# URL веб-страницы для поиска видеопотока
url = "http://webcam.exdesign.ru/zilart-16/"

# Вызов функции поиска URL видеопотока
stream_url = find_video_url(url)

# Вывод найденного URL или сообщения о том, что URL не найден
if stream_url:
    print("URL видеопотока:", stream_url)
else:
    print("URL видеопотока не найден.")

# Определение области интереса
#x0, y0, x1, y1 = 1, 1, 1200, 640
x0, y0, x1, y1 = 1015, 78, 1150, 175

# Пороговое значение и фильтрация шума
threshold_value = 25  # Исходное пороговое значение

# Telegram Bot Token и Chat ID
telegram_bot_token = ''
telegram_chat_id = ''

while True:
    try:
        print("Opening the video stream...")
        cap = cv2.VideoCapture(stream_url)

        # Инициализируем время последнего уведомления
        last_notification_time = time.time()

        while True:
            ret, frame1 = cap.read()
            gray1 = cv2.cvtColor(frame1[y0:y1, x0:x1], cv2.COLOR_BGR2GRAY)
            # Пауза в 5 секунд
            time.sleep(5)

            ret, frame2 = cap.read()
            gray2 = cv2.cvtColor(frame2[y0:y1, x0:x1], cv2.COLOR_BGR2GRAY)

            # Вычисляем абсолютную разницу между двуми кадрами
            diff = cv2.absdiff(gray1, gray2)

            # Применяем пороговое значение, чтобы получить бинарное изображение
            _, threshold = cv2.threshold(diff, threshold_value, 255, cv2.THRESH_BINARY)

            # Применяем фильтрацию шума
            threshold = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

            # Выделение движущихся объектов зеленым прямоугольником внутри области интереса
            contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            motion_objects = []
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame1, (x0 + x, y0 + y), (x0 + x + w, y0 + y + h), (0, 255, 0), 2)
                motion_objects.append((x0 + x, y0 + y, w, h))

            # Если на кадре обнаружено движение и прошло более 5 минут с последнего уведомления, выводим его и отправляем уведомление на электронную почту и Telegram
            if np.sum(threshold) > 0:
                print("Обнаружено движение!")
                frame1 = cv2.rectangle(frame1, (x0, y0), (x1, y1), (0, 0, 255), 2)  # Выделение области интереса
                clear_output(wait=True)
                frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
                display(Image.fromarray(frame))

                # Сохранение изображений до и после обнаружения движения
                current_time = time.strftime("%Y%m%d-%H%M%S")
                prev_frame_path = f"motion/prev_frame_{current_time}.jpg"
                current_frame_path = f"motion/current_frame_{current_time}.jpg"

                cv2.imwrite(prev_frame_path, frame1)
                cv2.imwrite(current_frame_path, frame2)

                print("Предыдущий кадр сохранен как:", prev_frame_path)
                print("Текущий кадр сохранен как:", current_frame_path)

                # Отправка уведомления в Telegram
                telegram_message = "Обнаружено движение на видеопотоке."
                send_telegram_notification(telegram_bot_token, telegram_chat_id, telegram_message, current_frame_path,
                                        x0, y0, x1, y1, motion_objects)

                last_notification_time = time.time()
                # Пауза в 5 секунд
                time.sleep(60)

            # Обновляем кадры
            frame1 = frame2
            ret, frame2 = cap.read()

    except Exception as e:
        print(f"Произошла ошибка: {e}")
        print("Перезапуск видеопотока...")
        cap.release()
        stream_url = find_video_url(url)
        continue

print("Видеопоток закрыт.")
